**Importation and Treatment of our data** 

In [52]:
import pandas as pd
from IPython.display import display, Math # to show latex
from matplotlib import pyplot as plt
import numpy as np
data = pd.read_csv("bank-additional-full.csv",delimiter=';')

In [3]:
filter1 = data['housing'] != 'unknown'
filter2= data['loan'] != 'unknown'
filter3 = data['education'] != 'unknown'
filter4 = data['job'] != 'unknown'
filter5 = data['default'] != 'unknown'

temp = data.where(filter1&filter2&filter3&filter4&filter5)
dow = {'mon':1,'tue':2,'wed':3,'thu':4,'fri':5}
mon = {"may":5,"jun":6,"jul":7,"aug":8,"oct":10,"nov":11,"dec":12,"mar":3,"apr":4,"sep":9}
temp = temp.dropna()
temp['education'] = temp.apply(lambda x: x['education'] in ['professional.course','university.degree'],axis=1)
temp['marital'] = temp.apply(lambda x:x['marital'] =='married',axis=1)
temp['poutcome'] = temp.apply(lambda x:x['poutcome'] =='success',axis=1)
temp['day_of_week'] = temp.apply(lambda x:dow[x['day_of_week']] ,axis=1)
temp['month'] = temp.apply(lambda x:mon[x['month']] ,axis=1)
temp['default'] = temp.apply(lambda x:x['default']=="yes" ,axis=1)
temp['housing'] = temp.apply(lambda x:x['housing']=="yes" ,axis=1)
temp['loan'] = temp.apply(lambda x:x['loan']=="yes" ,axis=1)
temp['y'] = temp.apply(lambda x:x['y']=="yes" ,axis=1)

#dropping unwanted columns
temp = temp.drop('contact', inplace=False, axis=1)
"""
temp = temp.drop('duration', inplace=False, axis=1)
temp = temp.drop('emp.var.rate', inplace=False, axis=1)
temp = temp.drop('cons.price.idx', inplace=False, axis=1)
temp = temp.drop('cons.conf.idx', inplace=False, axis=1)
temp = temp.drop('euribor3m', inplace=False, axis=1)
temp = temp.drop('nr.employed', inplace=False, axis=1)
"""
data = temp
#data

**Research Question :** What is the influence of the Age (Continuous variable), Pdays(Discrete variable) and Job(Discrete variable) values on the Duration value (Continuous variable) ?

In [4]:
data2 = data[["age","duration","pdays","job"]]
temp_data = data2[data2["pdays"]!= 999]
temp_data

,age,duration,pdays,job
24108,37.0,119.0,6.0,admin.
24264,48.0,112.0,4.0,management
24482,36.0,200.0,4.0,self-employed
24613,32.0,1091.0,5.0,blue-collar
24799,31.0,290.0,5.0,entrepreneur
...,...,...,...,...
41163,35.0,385.0,4.0,technician
41164,54.0,1868.0,10.0,admin.
41174,62.0,208.0,1.0,retired
41178,62.0,483.0,6.0,retired


**Part 1: Linear Regression**

In [5]:
import random
data_sample_200 =  temp_data.sample(200,random_state=1)

In [6]:
df = pd.DataFrame(data_sample_200, columns = ['job','duration','age','pdays'])
a = df.groupby(['job']).median().sort_values(['duration']).reset_index()
a= a['job'].to_numpy()

In [7]:
job_to_index = {}
for i,j in enumerate(a):
  job_to_index[j] = i

In [8]:
df = pd.DataFrame(data_sample_200, columns = ['job','duration','age','pdays'])
df2 = pd.DataFrame(temp_data, columns = ['job','duration','age','pdays'])
df2['temp'] = df2.apply(lambda y : job_to_index[y['job']],axis = 1)

In [9]:

df['temp'] = df.apply(lambda y : job_to_index[y['job']],axis = 1)

**Q.1:**

In [10]:
X = df[['temp','age','pdays']].to_numpy()
X = np.hstack((np.ones((len(df.index),1)),X))
Y = df['duration'].to_numpy()
beta_hat = np.linalg.inv(X.T@X)@X.T@Y
print("Beta hat values based on our sample are:", beta_hat)

Beta hat values based on our sample are: [317.19568028   8.33439449  -0.46539474  -1.06261204]


In [11]:
p = len(beta_hat)
P_big = X@np.linalg.inv(X.T@X)@X.T
k=p-1
SSE = Y.T @(np.identity(200)-P_big)@Y

**Q1.a**

In [12]:
dict_b = {}
sigma_data = SSE/196
C = np.linalg.inv(X.T@X)
for i in range(4):
  print(" For beta",i,"CI is:")
  dict_b[i] = [beta_hat[i] - 2*np.sqrt(sigma_data*C[i][i]),beta_hat[i] + 2*np.sqrt(sigma_data*C[i][i])]
  print(dict_b[i])
  #print('Lenght of the CI of beta', i,' based on the variance matrix is:',print)

 For beta 0 CI is:
[186.57323338410663, 447.8181271728637]
 For beta 1 CI is:
[-2.822088026536422, 19.490876999756722]
 For beta 2 CI is:
[-2.8426552537701157, 1.9118657661750929]
 For beta 3 CI is:
[-10.266015132186762, 8.140791060168315]


In [13]:
data_400 = df.sample(n=400, replace=True)
df_400 = pd.DataFrame(data_400, columns = ['job','duration','age','pdays'])
df_400['temp'] = df_400.apply(lambda y : job_to_index[y['job']],axis = 1)
X_b = df_400[['temp','age','pdays']].to_numpy()
X_b = np.hstack((np.ones((len(df_400.index),1)),X_b))
Y_b = df_400['duration'].to_numpy()

beta_hat_b = np.linalg.inv(X_b.T@X_b)@X_b.T@Y_b
beta_hat_b

array([372.10467284,   6.03882383,  -1.88942589,   1.5223421 ])

In [14]:
Beta_avg=0
sum=0
for i in beta_hat_b:
  Beta_avg = Beta_avg + (i**2)
  sum = sum + i
Beta_avg_2 = 1/400*Beta_avg
Beta_avg = (1/400*sum)**2
se_boot = np.sqrt(Beta_avg_2-Beta_avg)
se_boot

18.584096349874084

**Q1.b**

In [15]:
dict_c={}
for i in range(4):
  print(" For beta",i,"CI is:")
  dict_c[i] = [beta_hat_b[i] - 2*se_boot,beta_hat_b[i] + 2*se_boot]
  print(dict_c[i])

 For beta 0 CI is:
[334.9364801451193, 409.27286554461557]
 For beta 1 CI is:
[-31.12936886982775, 43.207016529668586]
 For beta 2 CI is:
[-39.057618589300844, 35.27876681019549]
 For beta 3 CI is:
[-35.64585060301952, 38.69053479647682]


**Q1.c**

In [16]:
dict_d={}
for i in range(4):
  print(" For beta",i,"CI is:")
  dict_d[i] = [2*beta_hat[i]-np.quantile(beta_hat_b[i],0.975),2*beta_hat[i]+np.quantile(beta_hat_b[i],0.025)]
  print(dict_d[i])

 For beta 0 CI is:
[262.2866877121029, 1006.4960334018377]
 For beta 1 CI is:
[10.629965143299883, 22.707612803140712]
 For beta 2 CI is:
[0.9586364019576539, -2.820215377147699]
 For beta 3 CI is:
[-3.647566168747099, -0.6028819752897978]


**Q1.d**

In [17]:
dict_e={}
for i in range(4):
  print(" For beta",i,"CI is:")
  dict_e[i] = [np.quantile(beta_hat_b[i],0.025),np.quantile(beta_hat_b[i],0.975)]
  print(dict_e[i])

 For beta 0 CI is:
[372.1046728448674, 372.1046728448674]
 For beta 1 CI is:
[6.038823829920416, 6.038823829920416]
 For beta 2 CI is:
[-1.8894258895526765, -1.8894258895526765]
 For beta 3 CI is:
[1.5223420967286505, 1.5223420967286505]


**Q.2**

Lenght of our confidence intervals:

In [51]:
for i in range(4):
  print('Lenght of the CI of beta', i,' based on the variance matrix:')
  print(abs(dict_b[i][1]-dict_b[i][0]+1))
  print('Lenght of the CI of beta', i,' based on normal distribution with Bootstrap:')
  print(abs(dict_c[i][1]-dict_c[i][0]+1))
  print('Lenght of the CI of beta', i,' based on pivot technique with Bootstrap:')
  print(abs(dict_d[i][1]-dict_d[i][0]+1))
  print('Lenght of the CI of beta', i,' based on percentile technique with Bootstrap:')
  print(abs(dict_e[i][1]-dict_e[i][0]+1))

Lenght of the CI of beta 0  based on the variance matrix:
262.24489378875705
Lenght of the CI of beta 0  based on normal distribution with Bootstrap:
75.3363853994963
Lenght of the CI of beta 0  based on pivot technique with Bootstrap:
745.2093456897348
Lenght of the CI of beta 0  based on percentile technique with Bootstrap:
1.0
Lenght of the CI of beta 1  based on the variance matrix:
23.312965026293142
Lenght of the CI of beta 1  based on normal distribution with Bootstrap:
75.33638539949634
Lenght of the CI of beta 1  based on pivot technique with Bootstrap:
13.07764765984083
Lenght of the CI of beta 1  based on percentile technique with Bootstrap:
1.0
Lenght of the CI of beta 2  based on the variance matrix:
5.754521019945209
Lenght of the CI of beta 2  based on normal distribution with Bootstrap:
75.33638539949634
Lenght of the CI of beta 2  based on pivot technique with Bootstrap:
2.778851779105353
Lenght of the CI of beta 2  based on percentile technique with Bootstrap:
1.0
Len

Verification: Is our CI contains our real values of beta ?

In [19]:
X2 = df2[['temp','age','pdays']].to_numpy()
X2 = np.hstack((np.ones((len(df2.index),1)),X2))
Y2 = df2['duration'].to_numpy()
beta_hat2 = np.linalg.inv(X2.T@X2)@X2.T@Y2

for i in range(4):
  print("Is beta",i ,"obtained from the original data is in our CI ? ")
  print((beta_hat2[i]>dict_b[i][0])&(beta_hat2[i] < dict_b[i][1]))

Is beta 0 obtained from the original data is in our CI ? 
True
Is beta 1 obtained from the original data is in our CI ? 
True
Is beta 2 obtained from the original data is in our CI ? 
True
Is beta 3 obtained from the original data is in our CI ? 
True


In [20]:
for i in range(4):
  print("Is beta",i ,"obtained from the original data is in our CI ? ")
  print((beta_hat2[i]>dict_c[i][0])&(beta_hat2[i] < dict_c[i][1]))

Is beta 0 obtained from the original data is in our CI ? 
False
Is beta 1 obtained from the original data is in our CI ? 
True
Is beta 2 obtained from the original data is in our CI ? 
True
Is beta 3 obtained from the original data is in our CI ? 
True


In [21]:
for i in range(4):
  print("Is beta",i ,"obtained from the original data is in our CI ? ")
  print((beta_hat2[i]>dict_d[i][0])&(beta_hat2[i] < dict_d[i][1]))

Is beta 0 obtained from the original data is in our CI ? 
True
Is beta 1 obtained from the original data is in our CI ? 
False
Is beta 2 obtained from the original data is in our CI ? 
False
Is beta 3 obtained from the original data is in our CI ? 
False


In [22]:
for i in range(4):
  print("Is beta",i ,"obtained from the original data is in our CI ? ")
  print((beta_hat2[i]>dict_e[i][0])&(beta_hat2[i] < dict_e[i][1]))

Is beta 0 obtained from the original data is in our CI ? 
False
Is beta 1 obtained from the original data is in our CI ? 
False
Is beta 2 obtained from the original data is in our CI ? 
False
Is beta 3 obtained from the original data is in our CI ? 
False


**Q.3**

In [23]:
df3 = temp_data.sample(300,random_state=33)
df3 = pd.concat([df3,data_sample_200]).drop_duplicates(keep=False).head(100)

In [24]:
df3['temp'] = df3.apply(lambda y : job_to_index[y['job']],axis = 1)
X_all = df3[['temp','age','pdays']].to_numpy()
X_all = np.hstack((np.ones((len(df3.index),1)),X_all))
Y_all = df3['duration'].to_numpy()

**Q3.a**

In [25]:
beta_hat3 = np.linalg.inv(X_all.T@X_all)@X_all.T@Y_all
y3_pred = beta_hat @ X_all.T
for i in range(100):
  print("Prediction given X_new_",i,"is:")
  print(y3_pred[i])

Prediction given X_new_ 0 is:
291.2035017243794
Prediction given X_new_ 1 is:
319.9026269377801
Prediction given X_new_ 2 is:
360.7681693754104
Prediction given X_new_ 3 is:
374.2565852968427
Prediction given X_new_ 4 is:
291.2035017243794
Prediction given X_new_ 5 is:
359.56570314229145
Prediction given X_new_ 6 is:
311.5799539430446
Prediction given X_new_ 7 is:
289.2181318494708
Prediction given X_new_ 8 is:
315.4349344418389
Prediction given X_new_ 9 is:
297.25363339374707
Prediction given X_new_ 10 is:
318.9563427452499
Prediction given X_new_ 11 is:
306.3523155664886
Prediction given X_new_ 12 is:
312.90621107588225
Prediction given X_new_ 13 is:
320.3525269766424
Prediction given X_new_ 14 is:
359.185730201984
Prediction given X_new_ 15 is:
311.2699081012921
Prediction given X_new_ 16 is:
346.1973882773988
Prediction given X_new_ 17 is:
356.9478681296656
Prediction given X_new_ 18 is:
341.3849706866556
Prediction given X_new_ 19 is:
360.5856042765556
Prediction given X_new_ 20 i

**Q3.b**

In [26]:
predictions_array = np.array(y3_pred)
expectation_CI = []

for i in range(100):
  pred = np.sort(predictions_array)
  ci = [pred[5], pred[95]]
  expectation_CI.append(ci)

for i in range(100):
  print("Confidence Interval for the expectation of Y_New based on X_New point",i,"is:")
  print(expectation_CI[i])


Confidence Interval for the expectation of Y_New based on X_New point 0 is:
[291.7660397635417, 372.26318377323844]
Confidence Interval for the expectation of Y_New based on X_New point 1 is:
[291.7660397635417, 372.26318377323844]
Confidence Interval for the expectation of Y_New based on X_New point 2 is:
[291.7660397635417, 372.26318377323844]
Confidence Interval for the expectation of Y_New based on X_New point 3 is:
[291.7660397635417, 372.26318377323844]
Confidence Interval for the expectation of Y_New based on X_New point 4 is:
[291.7660397635417, 372.26318377323844]
Confidence Interval for the expectation of Y_New based on X_New point 5 is:
[291.7660397635417, 372.26318377323844]
Confidence Interval for the expectation of Y_New based on X_New point 6 is:
[291.7660397635417, 372.26318377323844]
Confidence Interval for the expectation of Y_New based on X_New point 7 is:
[291.7660397635417, 372.26318377323844]
Confidence Interval for the expectation of Y_New based on X_New point 8 

**Q3.c**

In [27]:
sum=0
C3 = np.linalg.inv(X_all.T@X_all)
e = [a-b for a,b in zip(Y_all,y3_pred)]
n = len(df3.index)
for a in e:
  sum = a**2 +sum
sigma_3 =sum / (n-4)

In [28]:
CI_1000 = {}
rr = range(n)
for x,y,i in zip(X_all,y3_pred,rr):
  var_for_ci = 2*np.sqrt(sigma_3*(x@C3@x.T+1))
  CI_1000[i] = (y-var_for_ci,y+var_for_ci)

In [29]:
prob = 0
for i in range(100):
  if (Y_all[i] >= CI_1000[i][0]) and (Y_all[i] <= CI_1000[i][1]):
    prob +=1

prob /= 100
print("The percentage of confidence intervals that contains the real values of Y_new is:",prob)

The percentage of confidence intervals that contains the real values of Y_new is: 0.97


We note that the percentage obtained is close to 95%. It proves the validation of our confidence intervals that we built.

**Q3.d**

**Idea in order to calculate the confidence interval of Y_New based on Bootstrap:** We have to calculate the variance of (Y_New(estimated) + the noise epsilon) and the vector of Y_New estimated in order to compute the confidence interval based on Bootstrap.
We have already computhe the vector of Y_new estimated.
Thus, we can compute an estimator of the variance of Y_New estimated and the variance of the noise based on a simulation/estimator and therefore compute the CI asked.


**Part 2: Tests**

We consider the following research question from our project 2 : Does the number of contacts performed in this campaign and for a client ('campaign' label) influences the outcome of the marketing campaign ('y' label) ?

**Q.1**

Then our reformulated research question is : Is the distribution of the 'campaign' values when y == True is superior to the distribution of the 'campaign' values when y == False ? 

**Q.2**

In [30]:
data3 = data[["campaign","y"]]
data3_200 =  data3.sample(200,random_state=1)

**Q3.a**

In [31]:
data_false = data3_200.loc[data3_200['y'] == 0]['campaign'].values
data_true = data3_200.loc[data3_200['y'] == 1]['campaign'].values
mu_1 = data_true.mean()
mu_2 = data_false.mean()
delta_means = mu_1-mu_2
print('value of mu_1 is:',mu_1)
print('value of mu_2 is:',mu_2)
print('value of delta is:',delta_means)

value of mu_1 is: 2.269230769230769
value of mu_2 is: 2.4885057471264367
value of delta is: -0.21927497789566752


Based on what we learnt, the MLE for the expectation is the mean of our data. Thus, the MLE for a linear function of expectations (delta value) is this linear function applied on our MLE: in our case the difference between the 2 means.

**Q3.b**

In [32]:
std_1 = np.std(data_true)
std_2 = np.std(data_false)
std_delta = ((std_1 / len(data_false)) + (std_2 / len(data_true)))** 0.5
delta_CI = (delta_means - (2 * std_delta), delta_means + (2 * std_delta))

print("Confidence interval of delta is: "+ str(delta_CI))

Confidence interval of delta is: (-0.9675621746283543, 0.5290122188370192)


**Q3.c**

In [33]:
delta_all = data3[data3['y']==True]['campaign'].mean() - data3[data3['y']==False]['campaign'].mean()
delta_all

-0.5682432274504272

In [34]:
print("Is delta obtained from the original data is in our CI ? ")
print(delta_all>delta_CI[0] and delta_all<delta_CI[1])

Is delta obtained from the original data is in our CI ? 
True


**Q3.d**

In [35]:
import random
import math
from scipy.stats import stats

**Wald Test:**

In [76]:
display(Math(("""H_0 : \mu_1 - \mu_2 =0""")))
display(Math(("""H_1 : \mu_1 - \mu_2 != 0""")))
display(Math(("""W = \\frac{\mu_1-\mu_2}{\\sqrt{\\frac{S^2_1}{n_1}+\\frac{S_1^2}{n_2}}}""")))
display(Math(("""|W| > z_{\\alpha\over2}""")))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [36]:
from scipy.stats import norm

TS = delta_means / std_delta
dist_value = norm.ppf(1-0.05/2)
dist_cdf_value = norm.cdf(TS)
if TS > 0:
    p_value = 2*(1-dist_cdf_value)
else:
    p_value = dist_cdf_value
print('Wald test statistic is : ', str(TS))
rejection_area_Wald = np.abs(TS) > dist_value
print("Are we rejecting H0 ? ",rejection_area_Wald)  
print("p-value is : ", p_value)

Wald test statistic is :  -0.5860717084379031
Are we rejecting H0 ?  False
p-value is :  0.27891366370952275


**Permutations test:**

In [86]:
display(Math("""H_0: F_x = F_Y"""))
display(Math("""H_1:F_X \prec F_Y"""))
print("T.S is : W=mean1-mean2")
print("P-value is :")
display(Math("""\\frac{\sum \mathbb{I}(W_s^b \geq W_s^obs))}{B}"""))
print("R.R is :")
display(Math("""Pvalue < \\alpha"""))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

T.S is : W=mean1-mean2
P-value is :


<IPython.core.display.Math object>

R.R is :


<IPython.core.display.Math object>

In [37]:
import os 
import pandas as pd
import numpy as np
from scipy.stats import norm
from random import sample

In [38]:
def permutations_function(all_data, X_data, X_size, Y_data, Y_size, B):
    real_stat = 0
    permutations = []
    real_stat = np.mean(Y_data) - np.mean(X_data)
    for b in range(B):
        X_sample = sample(all_data,X_size)
        Y_sample = sample(all_data,Y_size)
        sample_stat = np.mean(Y_sample) - np.mean(X_sample)
        permutations.append(sample_stat)
    return real_stat, permutations

In [39]:
B = 400
X_data = data_true.tolist().copy()
X_size = len(X_data)
Y_data = data_false.tolist().copy()
Y_size = len(Y_data)
all_data = X_data + Y_data

mean_diff, permutations = permutations_function(all_data, X_data, X_size, Y_data, Y_size, B)
indic = 0
for mean in permutations:
    if mean >= mean_diff:
        indic = indic + 1
p_value = indic / B

print('p-value is:', p_value)
rejection_area = p_value < 0.05
print("Are we rejecting H0 ? ",rejection_area)  

p-value is: 0.3925
Are we rejecting H0 ?  False


**Q4.a**

In [40]:
m1= np.median(data_true)
m2= np.median(data_false)
print('value of median1 is:',m1)
print('value of median2 is:',m2)
print('value of delta is:',m1-m2)

value of median1 is: 1.5
value of median2 is: 1.0
value of delta is: 0.5


The estimator obtained isn't a MLE based on what we learn.

**Q4.b**

In [41]:
se_1 = np.std(data_true)
se_2 = np.std(data_false)

delta_medians = ((se_1 / len(data_true)) + (se_2 / len(data_false)))** 0.5
medians_delta_CI = (delta_medians - (2 * delta_medians), delta_medians + (2 * delta_medians))

print("CI of delta medians is: ",(medians_delta_CI))

CI of delta medians is:  (-0.3129695731410684, 0.9389087194232051)


**Q4.c**

In [42]:
delta_medians_all = data3[data3['y']==True]['campaign'].median() - data3[data3['y']==False]['campaign'].median()
delta_medians_all

0.0

In [43]:
print("Is delta obtained from the original data is in our CI ? ")
print(delta_medians_all>medians_delta_CI[0] and delta_medians_all<medians_delta_CI[1])

Is delta obtained from the original data is in our CI ? 
True


**Q4.d**

**Permutations test:**

In [87]:
display(Math("""H_0: F_x = F_Y"""))
display(Math("""H_1:F_X \prec F_Y"""))
print("T.S is : W=median1-median2")
print("P-value is :")
display(Math("""\\frac{\sum \mathbb{I}(W_s^b \geq W_s^obs))}{B}"""))
print("R.R is :")
display(Math("""Pvalue < \\alpha"""))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

T.S is : W=median1-median2
P-value is :


<IPython.core.display.Math object>

R.R is :


<IPython.core.display.Math object>

In [44]:
def permutations_function_medians(all_data, X_data, X_size, Y_data, Y_size, B):
    real_stat = 0
    permutations = []
    real_stat = np.median(Y_data) - np.median(X_data)
    for b in range(B):
        X_sample = sample(all_data,X_size)
        Y_sample = sample(all_data,Y_size)
        sample_stat = np.median(Y_sample) - np.median(X_sample)
        permutations.append(sample_stat)
    return real_stat, permutations

In [45]:
B = 400
X_data = data_true.tolist().copy()
X_size = len(X_data)
Y_data = data_false.tolist().copy()
Y_size = len(Y_data)
all_data = X_data + Y_data

median_diff, permutations = permutations_function_medians(all_data, X_data, X_size, Y_data, Y_size, B)
indic = 0
for median in permutations:
    if median >= median_diff:
        indic = indic + 1
p_value = indic / B

print('p-value is:', p_value)
rejection_area = p_value < 0.05
print("Are we rejecting H0 ? ",rejection_area) 

p-value is: 0.7875
Are we rejecting H0 ?  False


**Q4.e**

Based on what we learnt, the assumption of asymptotic normality is necessary for the application of Wald or t tests. However, in this case, the estimator is not a maximum likelihood estimator. Thus we cannot use the Wald/t tests in our case.

**Q.5:**

In [82]:
display(Math("""H_0: F_x = F_Y"""))
display(Math("""H_1:F_X \prec F_Y"""))
print("T.S is :")
display(Math(""" W_s = \sum_{i=1}^{n}S_i"""))
print("P-value is :")
display(Math("""\\frac{\sum \mathbb{I}(W_s^b \geq W_s^obs))}{B}"""))
print("R.R is :")
display(Math("""Pvalue < \\alpha"""))


<IPython.core.display.Math object>

<IPython.core.display.Math object>

T.S is :


<IPython.core.display.Math object>

P-value is :


<IPython.core.display.Math object>

R.R is :


<IPython.core.display.Math object>

In [46]:
arr = data3_200.to_numpy()
arr2 = sorted(arr,key = lambda x:x[0])
arr3 = [[i+1,x] for i,x in enumerate(arr2)]
arr4 = []
for el in arr3:
  sim = [x for x in arr3 if x[1][0]==el[1][0]]
  arr4.append([np.mean([x[0] for x in sim]),el[1]])

In [47]:
s_statistic =np.sum([x[0] for x in arr4 if x[1][1]==True])

In [49]:
num_of_rounds = 20000
counter = 0
for i in range(num_of_rounds):
  temp_sample = np.random.choice([x[0] for x in arr4],data_true.shape[0])
  s_stat_midgam = np.sum(temp_sample)
  indic = 1 if s_stat_midgam > s_statistic else 0
  counter +=indic

p_val = counter / num_of_rounds
print('p-value is:', p_val)
rejection_area = p_val < 0.05
print("Are we rejecting H0 ? ",rejection_area) 

p-value is: 0.5359
Are we rejecting H0 ?  False


**Q.6:**

In question 3, we used two different tests (Wald and permutation) to show if the mean across the two populations is the same or not. In both cases, we were unable to reject the null hypothesis.

In the same way, in question 4, we demonstrated (using permutation test) that the median across the two populations is likely the same as well, as we were unable to reject the null hypothesis.

In question 5, we showed that 
F(X) ≺ F(Y). Thus although the mean and median are the same, the distribution is different for the 2 categories and there is a distribution superior.